# Project: Data jobs posted in 2023

Questions I wish to answer:
    
1. What are the top-paying Data Analyst jobs?
2. What skills are required for the top-paying jobs identified in question 1?
3. What are the most in-demand skills for Data Analyst jobs?
4. What are the top skills based on salary for Data Analyst jobs?
5. What are the most optimal skills to learn? i.e. high demand and high paying

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# load data
company_dim = pd.read_csv('company_dim.csv')
job_postings_fact = pd.read_csv('job_postings_fact.csv')
skills_dim = pd.read_csv('skills_dim.csv')
skills_job_dim = pd.read_csv('skills_job_dim.csv')

### Explore job_postings_fact

In [3]:
job_postings_fact.head(3)

,job_id,company_id,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg
0,0,0,Data Analyst,Marketing Data Analyst,Anywhere,via LinkedIn,Full-time,True,Serbia,2023-09-25 17:46:06,False,False,Serbia,NaN,NaN,NaN
1,55,1,Cloud Engineer,Storage and Virtualization Engineer,"Kuwait City, Kuwait",via Trabajo.org,Full-time,False,Kuwait,2023-07-30 17:49:18,True,False,Kuwait,NaN,NaN,NaN
2,66,2,Data Analyst,Data Analyst et Scientist F/H,"Paris, France",via Emplois Trabajo.org,Full-time,False,France,2023-07-28 17:28:01,False,False,France,NaN,NaN,NaN


In [4]:
# number of job listings
job_postings_fact.shape[0]

787686

In [5]:
# check for missing data
for col in job_postings_fact.columns:
    NoData = np.mean(job_postings_fact[col].isnull()) * 100
    print(f'{col} : {NoData}%')

job_id : 0.0%
company_id : 0.0%
job_title_short : 0.0%
job_title : 0.00012695414162496222%
job_location : 0.13368271113108524%
job_via : 0.00114258727462466%
job_schedule_type : 1.6129523693451453%
job_work_from_home : 0.0%
search_location : 0.0%
job_posted_date : 0.0%
job_no_degree_mention : 0.0%
job_health_insurance : 0.0%
job_country : 0.006601615364498037%
salary_rate : 95.79781791221374%
salary_year_avg : 97.20269244343558%
salary_hour_avg : 98.64603407956977%


In [6]:
set(job_postings_fact['salary_rate'])

{'day', 'hour', 'month', nan, 'week', 'year'}

### Observations:
- The majority of job listing don't include salary information
- Possible salary rates include day, week and month, but there are no corresponding salary value columns for these rates? Need to investigate this further.

In [7]:
job_postings_fact.groupby('salary_rate', as_index=False)['job_id'].count()

,salary_rate,job_id
0,day,10
1,hour,10665
2,month,379
3,week,11
4,year,22035


In [8]:
job_postings_fact.groupby('salary_rate', as_index=False)[['salary_year_avg', 'salary_hour_avg']].count()

,salary_rate,salary_year_avg,salary_hour_avg
0,day,0,0
1,hour,0,10665
2,month,0,0
3,week,0,0
4,year,22034,0


### Observations:
- For the 787.686 jobs in the dataset, we have salary rate data for 33.100 job.
- For these 33.100 jobs we only have salary value data for 22.034 jobs with a yearly salary specified and 10.665 jobs with an hourly salary rate specified.
- It is unclear from the dataset how many hours workers in the jobs with hourly rates specified would be able to work in a year. A direct comparison of the salaries from jobs with yearly salary rates and hourly salary rates is therefore not possible.
- For the purposes of my analysis I will therefore only focus on the 22.034 jobs with yearly salary information recorded.

### Explore company_dim

In [9]:
company_dim.head()

,company_id,name,link,link_google,thumbnail
0,0,Cryptology,NaN,https://www.google.com/search?sca_esv=56842508...,https://encrypted-tbn0.gstatic.com/images?q=tb...
1,1,Edraak,NaN,https://www.google.com/search?hl=en&gl=us&q=Ed...,https://encrypted-tbn0.gstatic.com/images?q=tb...
2,2,Groupe ADP,http://www.groupeadp.fr/,https://www.google.com/search?gl=us&hl=en&q=Gr...,https://encrypted-tbn0.gstatic.com/images?q=tb...
3,3,Interactive Resources - iR,NaN,https://www.google.com/search?q=Interactive+Re...,https://encrypted-tbn0.gstatic.com/images?q=tb...
4,4,Cybernetic Search,NaN,https://www.google.com/search?gl=us&hl=en&q=Cy...,https://encrypted-tbn0.gstatic.com/images?q=tb...


### Explore skills_dim

In [10]:
skills_dim.head()

,skill_id,skills,type
0,0,sql,programming
1,1,python,programming
2,2,nosql,programming
3,3,scala,programming
4,4,java,programming


### Explore skills_job_dim

In [11]:
skills_job_dim.head()

,job_id,skill_id
0,0,0
1,0,1
2,66,1
3,76,0
4,76,1


### Combine the tables

In [12]:
# create a dataframe with job_id and skills
df_skills = pd.merge(skills_job_dim, skills_dim, on='skill_id', how='left')[['job_id','skill_id','skills']]

In [13]:
# create a dataframe with job posting details and company name 
df_jobs = pd.merge(job_postings_fact, company_dim, on='company_id', how='left')[['job_id', 'job_work_from_home','job_title_short', 'company_id', 'name', 'job_country', 'salary_year_avg']]

My analysis will focus on Data Analyst jobs that allow remote work.

In [14]:
# filter the df_jobs dataframe to only include data analysis jobs that allow remote work
df_jobs_A = df_jobs.loc[(df_jobs['job_title_short'] == 'Data Analyst') & (df_jobs['job_work_from_home'] == True)]

In [16]:
# create another version of df_jobs but only including listings with salary data
df_jobs_sal_A = df_jobs_A.loc[~df_jobs_A['salary_year_avg'].isna()]

### Question 1: What are the top-paying Data Analyst jobs?
- I will focus on the top 10 Data Analyst jobs allowing for remote work.
- I want to display the company name in the output as apposed to just a company id.

In [17]:
# filter out jobs with no salary data, sort,and limit output to top 10
df_q1 = df_jobs_sal_A.sort_values(by='salary_year_avg', ascending=False).head(10)

In [18]:
df_q1

,job_id,job_work_from_home,job_title_short,company_id,name,job_country,salary_year_avg
319998,226942,True,Data Analyst,319998,Mantys,India,650000.0
632984,547382,True,Data Analyst,21,Meta,United States,336500.0
157440,552322,True,Data Analyst,5835,AT&T,United States,255829.5
183972,99305,True,Data Analyst,183972,Pinterest Job Advertisements,United States,232423.0
381287,1021647,True,Data Analyst,381287,Uclahealthcareers,United States,217000.0
212796,168310,True,Data Analyst,19724,SmartAsset,United States,205000.0
333232,731368,True,Data Analyst,1261,Inclusively,United States,189309.0
364104,310660,True,Data Analyst,4707,Motional,United States,189000.0
180138,1749593,True,Data Analyst,19724,SmartAsset,United States,186000.0
670882,387860,True,Data Analyst,2686,Get It Recruit - Information Technology,United States,184000.0


### Question 2: What skills are required for the top-paying jobs identified in question 1?

In [19]:
df_q2 = pd.merge(df_q1, df_skills, on='job_id', how='left')

In [20]:
df_q2

,job_id,job_work_from_home,job_title_short,company_id,name,job_country,salary_year_avg,skill_id,skills
0,226942,True,Data Analyst,319998,Mantys,India,650000.0,NaN,NaN
1,547382,True,Data Analyst,21,Meta,United States,336500.0,NaN,NaN
2,552322,True,Data Analyst,5835,AT&T,United States,255829.5,0.0,sql
3,552322,True,Data Analyst,5835,AT&T,United States,255829.5,1.0,python
4,552322,True,Data Analyst,5835,AT&T,United States,255829.5,5.0,r
...,...,...,...,...,...,...,...,...,...
63,1749593,True,Data Analyst,19724,SmartAsset,United States,186000.0,182.0,tableau
64,1749593,True,Data Analyst,19724,SmartAsset,United States,186000.0,220.0,gitlab
65,387860,True,Data Analyst,2686,Get It Recruit - Information Technology,United States,184000.0,0.0,sql
66,387860,True,Data Analyst,2686,Get It Recruit - Information Technology,United States,184000.0,1.0,python


### Question 3: What are the most in-demand skills for Data Analyst jobs?
- Note: We are not concerned over salary here, so we will look at all job postings for data analyst positions i.e. not only those with yearly salary data.
- We need to check the number of job postings per skill and then sort the resulting list from highest to lowest. We will focus on the top 5 skills.

In [21]:
# combine job data with skills data
df_q3 = pd.merge(df_jobs_A, df_skills, on='job_id', how='left')

In [22]:
# Group combined data set by skills and count jobs. Sort number of jobs in decending order.
df_q3 = df_q3.groupby('skills', as_index=False)['job_id'].count().sort_values(by='job_id', ascending=False)

In [23]:
df_q3.head(5)

,skills,job_id
160,sql,7291
48,excel,4611
127,python,4330
170,tableau,3745
121,power bi,2609


### Question 4: What are the top skills based on salary for Data Analyst jobs?
- Need to calculate the average salary per skill and then find the top skills based on salary.

In [24]:
# combine job data with skills data
df_q4 = pd.merge(df_jobs_sal_A, df_skills, on='job_id', how='left')

In [25]:
df_q4 = df_q4.groupby('skills', as_index=False)['salary_year_avg'].mean().sort_values('salary_year_avg', ascending=False)[['skills', 'salary_year_avg']]

In [26]:
df_q4.head(25)

,skills,salary_year_avg
72,pyspark,208172.250000
8,bitbucket,189154.500000
111,watson,160515.000000
17,couchbase,160515.000000
21,datarobot,155485.500000
33,gitlab,154500.000000
100,swift,153750.000000
44,jupyter,152776.500000
62,pandas,151821.333333
24,elasticsearch,145000.000000


### Question 5: What are the most optimal skills to learn? i.e. high demand and high paying
- Will explore:
    - highest payed skills that have more than 10 job postings
    - average salary for most in-demand skills

In [27]:
df_q5 = pd.merge(df_jobs_sal_A, df_skills, on='job_id', how='left')

In [28]:
df_q5 = df_q5.groupby('skills', as_index=False)['job_id'].count().sort_values('job_id', ascending=False)[['skills', 'job_id']]

In [29]:
df_q5 = df_q5.loc[df_q5['job_id'] > 10]

In [30]:
df_q5 = pd.merge(df_q5, df_q4, on='skills', how='inner')[['skills', 'job_id', 'salary_year_avg']]

In [31]:
df_q5.sort_values(by='salary_year_avg', ascending=False)

,skills,job_id,salary_year_avg
17,go,27,115319.888889
34,confluence,11,114209.909091
20,hadoop,22,113192.568182
11,snowflake,37,112947.972973
13,azure,34,111225.102941
30,bigquery,13,109653.846154
15,aws,32,108317.296875
25,java,17,106906.441176
33,ssis,12,106683.333333
21,jira,20,104917.900000
